In [250]:
import numpy as np
import pandas as pd
from importlib import reload
from pylab import plt
from sklearn.preprocessing import PowerTransformer

import utils; reload(utils)

<module 'utils' from '/Users/kizawamasakazu/projects/machine_learning/work/time_series/stock_price_prediction/n225/utils.py'>

In [232]:
# データを準備
df = pd.read_csv('N225.csv')
df['log_return'] = np.log(df['Close']).diff()
# df['log_return'] = df['log_return'] / df['log_return'].rolling(30).std()
df['y'] = df['log_return'].shift(-1)
df.reset_index(inplace=True)

In [241]:
import torch
import torch.nn as nn
import torch.optim as optimizers
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

class MyDataset(Dataset):
    
    def __init__(self, X, y):
        assert X.shape[0] == y.shape[0]
        X = torch.tensor(X, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.float32)
        self.X = X
        self.y = y
        
    def __len__(self):
        return self.X.shape[0]
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
    

seqlen = 60
Xs, ys = [], []
df = df.dropna()
X = df['log_return'].values
y = df['y'].values
for i in range(X.shape[0]-seqlen+1):
    Xs.append(X[i:i+seqlen].reshape(1, -1))
    ys.append(y[i+seqlen-1])

Xs = np.stack(Xs)
ys = np.stack(ys)
print(Xs.shape)

train_val_X, test_X, train_val_y, test_y = train_test_split(Xs, ys, test_size=0.2, shuffle=False)
train_X, val_X, train_y, val_y = train_test_split(train_val_X, train_val_y, test_size=0.2, shuffle=False)

train_dataset = MyDataset(train_X, train_y)
val_dataset = MyDataset(val_X, val_y)

class CNN1d(nn.Module):
    
    def __init__(self, seqlen, n_filters = 64):
        super().__init__()
        cnn_output_size = int((int((seqlen - 10 + 1) / 2) - 5 + 1) / 2) * n_filters
        self.l1 = nn.Conv1d(1, n_filters, kernel_size=10)
        self.m1 = nn.MaxPool1d(kernel_size=2)
        self.a1 = nn.ReLU()
        self.l2 = nn.Conv1d(n_filters, n_filters, kernel_size=5)
        self.m2 = nn.MaxPool1d(kernel_size=2)
        self.a2 = nn.ReLU()
        self.f2 = nn.Flatten()
        self.l3 = nn.Linear(cnn_output_size, 1)
    
    def forward(self, x):
        x = self.l1(x)
        x = self.m1(x)
        x = self.a1(x)
        x = self.l2(x)
        x = self.m2(x)
        x = self.a2(x)
        x = self.f2(x)
        x = self.l3(x)
        return x
    
# test
model = CNN1d(seqlen=60)

(14432, 1, 60)


In [234]:
train_dataset[0][1].shape

torch.Size([])

In [248]:
epochs = 10

model = CNN1d(seqlen=60)
criterion = nn.L1Loss()
optimizer = optimizers.Adam(model.parameters(), lr=0.0005)
train_dataloader = DataLoader(train_dataset, batch_size=64)

def compute_loss(y, yhat):
    return criterion(yhat, y)

def train_step(x, y):
    model.train()
    yhat = model(x)
    loss = criterion(y, yhat)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return yhat, loss
    

for epoch in range(epochs):
    train_loss = 0.0
    for x, y in train_dataloader:
        _, loss = train_step(x, y)
        train_loss += loss.item()
    train_loss /= len(train_dataloader)
    print(f'epoch: {epoch}, train loss: {train_loss}')

epoch: 0, train loss: 0.008840788467305488
epoch: 1, train loss: 0.008158737133610351
epoch: 2, train loss: 0.008030212771725552
epoch: 3, train loss: 0.008094831267050628
epoch: 4, train loss: 0.008174248298244743
epoch: 5, train loss: 0.008101639315357496
epoch: 6, train loss: 0.008051190324577278
epoch: 7, train loss: 0.008045929877472848
epoch: 8, train loss: 0.008018700214873614
epoch: 9, train loss: 0.00799911745727576


In [246]:
class LSTM(nn.Module):
    
    def __init__(self, input_size, hidden_size, output_size=1, num_layers=1):
        super().__init__()
        self.l1 = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.l2 = nn.Linear(hidden_size, output_size)
        
        nn.init.xavier_normal_(self.l1.weight_ih_l0)
        nn.init.orthogonal(self.l1.weight_hh_l0)
        
    def forward(self, x):
        h, _ = self.l1(x)
        y = self.l2(h[:,-1])
        return y

model = LSTM(input_size=1, hidden_size=64)
model(torch.randn(10, 60, 1))

/var/folders/sv/k4676nzx20ndpwkjt51ctv380000gn/T/ipykernel_29120/3073272558.py:9: UserWarning: nn.init.orthogonal is now deprecated in favor of nn.init.orthogonal_.
  nn.init.orthogonal(self.l1.weight_hh_l0)


tensor([[-0.0623],
        [-0.0794],
        [-0.0690],
        [-0.0825],
        [-0.0684],
        [-0.0736],
        [-0.0769],
        [-0.0812],
        [-0.0777],
        [-0.0688]], grad_fn=<AddmmBackward0>)

In [249]:
epochs = 10

model = LSTM(input_size=1, hidden_size=64, output_size=1)

criterion = nn.L1Loss()
optimizer = optimizers.Adam(model.parameters(), lr=0.001)
train_dataloader = DataLoader(train_dataset, batch_size=64)

def compute_loss(y, yhat):
    return criterion(yhat, y)

def train_step(x, y):
    model.train()
    yhat = model(x)
    loss = criterion(y, yhat)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return yhat, loss
    

for epoch in range(epochs):
    train_loss = 0.0
    for x, y in train_dataloader:
        x = x.permute(0, 2, 1) # seq, inp, chanに並べ替え
        _, loss = train_step(x, y)
        train_loss += loss.item()
    train_loss /= len(train_dataloader)
    print(f'epoch: {epoch}, train loss: {train_loss}')

/var/folders/sv/k4676nzx20ndpwkjt51ctv380000gn/T/ipykernel_29120/3073272558.py:9: UserWarning: nn.init.orthogonal is now deprecated in favor of nn.init.orthogonal_.
  nn.init.orthogonal(self.l1.weight_hh_l0)


epoch: 0, train loss: 0.009009669745212485
epoch: 1, train loss: 0.008086812444801987
epoch: 2, train loss: 0.008154587441220365
epoch: 3, train loss: 0.008126327824168679
epoch: 4, train loss: 0.008100073106972309
epoch: 5, train loss: 0.00813271560192365
epoch: 6, train loss: 0.008112433904396563
epoch: 7, train loss: 0.008062213012177882
epoch: 8, train loss: 0.008053070844697027
epoch: 9, train loss: 0.00803957673688901
